In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb
from monai.networks.nets import UNet as MUNet

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "c:\Users\Brico\miniconda3\envs\monai\python.exe"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: Le module spécifié est introuvable.


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
def out_to_seg(img, num_classes):
    out = torch.zeros((1,*img.shape[1:])).to(img.device)
    for i,layer in enumerate(img) :
        out += layer*(i*255/(num_classes-1))
    return out

In [10]:
from numpy import isnan


def runTraining():
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 16
    batch_size_val = 8
    lr =   0.001  # Learning Rate
    epoch = 50 # Number of epochs
    
    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                      root_dir,
                                                      transform=transform,
                                                      mask_transform=mask_transform,
                                                      augment=True,
                                                      equalize=True)

    train_loader_full = DataLoader(train_set_full,
                              batch_size=batch_size,
                              worker_init_fn=np.random.seed(0),
                              num_workers=0,
                              shuffle=True)


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,
                            batch_size=batch_size_val,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)


    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    modelName = 'Test_Model'
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    # net = UNet(num_classes)
    net=MUNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=2,
        channels=(4, 8, 16, 32, 64, 128),
        strides=(1, 1, 1, 2, 2),
    )
    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax()
    CE_loss = torch.nn.MSELoss( reduction="sum")

    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

    ## DEFINE YOUR OPTIMIZER
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    BestEpoch = 0
    
    directory = 'Results/Statistics/' + modelName

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(epoch):
        net.train()
        lossEpoch = []
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, img_names = data

            ### From numpy to torch variables
            labels = to_var(labels)
            images = to_var(images)
            
            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = softMax(net(images))
            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            encoded_classes = batch_one_hot_encode_2_chan(segmentation_classes, 2)
            encoded_classes[:,1,:,:]=encoded_classes[:, -3:, :, :].sum(dim=1)
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(net_predictions, encoded_classes[:,:2, :, :]) # XXXXXX and YYYYYYY are your inputs for the CE
            # element_by_layers = predToSegmentation(net_predictions).sum(dim=(2,3))
            # prob_by_layer = (element_by_layers/(element_by_layers.sum()))
            # piecewise_diff = prob_by_layer - torch.Tensor([0.8,0.2]).to(prob_by_layer.device)
            # shannon = torch.pow(piecewise_diff, 2)
            lossTotal = CE_loss_value
            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            CE_loss_value.backward()
            optimizer.step()
            # THIS IS JUST TO VISUALIZE THE TRAINING 
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i) ,
                            length=15,
                             suffix=" Loss: {:.4f}, ".format(lossTotal))
            labels.cpu()
            images.cpu()

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)

        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch))

        
        ## THIS IS HOW YOU WILL SAVE THE TRAINED MODELS AFTER EACH EPOCH. 
        ## WARNING!!!!! YOU DON'T WANT TO SAVE IT AT EACH EPOCH, BUT ONLY WHEN THE MODEL WORKS BEST ON THE VALIDATION SET!!
        if not os.path.exists('./models/' + modelName):
            os.makedirs('./models/' + modelName)
        if i == epoch-1:
            torch.save(net.state_dict(), './models/' + modelName + '/' + str(i) + '_Epoch')
            
        np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)
    return net


In [11]:
testnet = runTraining()

----------------------------------------
~~~~~~~~  Starting the training... ~~~~~~
----------------------------------------
 Dataset: ./Data/ 
~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~
 Model Name: Test_Model
Total params: 166,104
~~~~~~~~~~~ Starting the training ~~~~~~~~~~
[Training] Epoch: 0 [DONE]                                      
[Training] Epoch: 0, LossG: 547223.4375                                                                      
[Training] Epoch: 1 [DONE]                                      
[Training] Epoch: 1, LossG: 523626.6250                                                                      
[Training] Epoch: 2 [DONE]                                      
[Training] Epoch: 2, LossG: 518783.9688                                                                      
[Training] Epoch: 3 [DONE]                                      
[Training] Epoch: 3, LossG: 517136.9375                                                                      
[Training] Epoch: 4

In [12]:
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
# mod = UNet(num_classes=4).cuda()
# mod.load_state_dict(torch.load("models/Test_Model/49_Epoch"))
# mod=testnet

transform = transforms.Compose([
        transforms.ToTensor()
    ])

In [9]:
# torch.cuda.empty_cache()
val_set = medicalDataLoader.MedicalImageDataset('train',
                                                    "./Data/",
                                                    transform=transform,
                                                    mask_transform=transform,
                                                    equalize=False)

val_loader = DataLoader(val_set,
                            batch_size=2,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)
# inference(mod, val_loader, "Test_Model", 99)

In [22]:
img, mask, path = val_set[10]
out = mod(to_var(img)[None,:])
out_cpu =out.detach().cpu()[0]
out_cpu = torch.nn.functional.softmax(out_cpu, dim=0)
segmentation_image = out_to_seg(predToSegmentation(out_cpu),3)

# Now, concatenated_output has shape (batch_size, 1, num_classes, height, width)
# If you want to remove the singleton dimension, you can use squeeze
print(out_cpu.max(dim=0))
fig, plots = plt.subplots(1,2, figsize=(20,20))
plots[0].imshow(segmentation_image.permute(1,2,0), cmap="gray")
plots[1].imshow(mask.permute(1,2,0), cmap="gray")
mask.unique(), path


In [24]:
val_set[:10]

ValueError: too many values to unpack (expected 2)